In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import subprocess
from multiprocessing import Pool
import re
from scipy import signal
from scipy.interpolate import interp1d

In [5]:
%run ./ltspice_helper.ipynb

In [6]:
def SGA (Xs, Xs_mult, Xs_names, gens, k, mu, max_inc, n_params, limits, granularity, mut_rate, runs, circ_name, target, heuristic):
    
    parents = np.zeros((k, n_params, runs))

    p1 = np.zeros((n_params,runs))
    p2 = np.zeros((n_params,runs))

    bestX = np.zeros((n_params,runs))
    bestS = 1e6*np.ones((runs,))

    Jhist = np.zeros((gens+1,runs))

    scores = np.zeros((mu,runs))
    
    for gen in range(0, gens):
        print("Gen: ", str(gen+1), "/", str(gens))
        
        print("Create Gen")
        for run in range(runs):
            lt_create_gen(circ_name, Xs[:, :, run], Xs_mult, Xs_names, run)
        
        results = []
        print("Simul")
        for run in range(runs):
            results.append(lt_simulate(circ_name, run))
        
        for p in results:
            if p.wait() != 0:
                print("There was an error")
        
        print("Score")
        scores = lt_score(circ_name, runs, mu, target)
        
        print("Mut")
        for run in range(runs):
            for i in range(0, mu):
                if scores[i, run] < bestS[run]:
                    bestS[run] = scores[i, run]
                    bestX[:,run] = Xs[i, :, run].copy()

            maxIdx = np.argmin(scores[:,run])
            Jhist[gen, run] = scores[maxIdx, run]

            topIdx = np.argsort(scores[:, run])
            for i in range(0, k):
                for j in range(0, n_params):
                    parents[i, j, run] = Xs[topIdx[i], j, run]

            for i in range(0, mu):
                r1 = np.random.randint(0, k)
                r2 = np.random.randint(0, k)

                for j in range(0, n_params):
                    Xs[i, j, run] = round(parents[r1, j, run]/2 + parents[r2, j, run]/2, 0)

            for i in range(0, mu):
                for j in range(0, n_params):
                    if np.random.rand() <= mut_rate:
                        increment = np.random.randint(-max_inc[j]/granularity[j], max_inc[j]/granularity[j])*granularity[j]
                        prevW = 0
                        if j >= n_params/2:
                            prevW = Xs[i, j-3, run]/Xs[i, j, run]
                        
                        Xs[i, j, run] += increment
                        if Xs[i, j, run] < limits[j, 0] or Xs[i, j, run] > limits[j, 1]:
                            Xs[i, j, run] -= 2*increment
                        if heuristic and j >= n_params/2:
                            Xs[i, j-3, run] = min(max(prevW*Xs[i, j, run], limits[j-3, 0]), limits[j-3, 1])
                            
    for run in range(runs):
        lt_create_gen(circ_name, Xs[:, :, run], Xs_mult, Xs_names, run)

    results = []
    for run in range(runs):
        results.append(lt_simulate(circ_name, run))

    for p in results:
        if p.wait() != 0:
            print("There was an error")
    
#     scores = lt_score_cutoff(circ_name, runs, mu, target)
    scores = lt_score(circ_name, runs, mu, target)
    for run in range(runs):
        Jhist[gens, run] = Jhist[gens-1, run]
        for i in range(0, mu):
            if scores[i, run] < bestS[run]:
                bestS[run] = scores[i,run]
                Jhist[gens, run] = scores[i,run]
                bestX[:, run] = Xs[i, :, run].copy()
    
    return bestS, bestX, Jhist